<a href="https://colab.research.google.com/github/Anaphase21/LoRA-fine-tuning/blob/main/LoRA_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wisdom Aduah**



#Imports

In [1]:
!pip install -q loralib
!pip install datasets --upgrade
!pip install livelossplot
!pip install colorama  # print colors :).
!pip install transformers -U
!pip install bert_score
!pip install evaluate
!pip install sacrebleu

!pip install datasets
!pip install peft
!pip accelerate

import os
import math
import urllib.request

# https://stackoverflow.com/questions/68340858/in-google-colab-is-there-a-programing-way-to-check-which-runtime-like-gpu-or-tpu
# if os.environ["COLAB_GPU"] and int(os.environ["COLAB_GPU"]) > 0:
#     print("a GPU is connected.")
# else:
#     print("Only CPU accelerator is connected.")

# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html#gpu-memory-allocation
# Avoid GPU memory allocation to be done by JAX.
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"

# note that jax might not run on a GPU here in colab, however we are not actually training jax models in this notebook
import chex
import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import optax

import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel
import datasets
import peft

from PIL import Image
from livelossplot import PlotLosses

# Utils.
import colorama

import torch
#import torchvision

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import itertools
import random

import copy

import gensim
from nltk.data import find
import nltk

nltk.download("word2vec_sample")

import huggingface_hub
import ipywidgets as widgets
from IPython.display import display

from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

from bert_score import score
from datasets import load_dataset, concatenate_datasets
from huggingface_hub import notebook_login
from IPython.display import display, Markdown

import evaluate


!pip install livelossplot
!pip install colorama  # print colors :).
! pip install sacrebleu
! pip install evaluate

#!pip install transformers datasets
#!pip install seaborn umap-learn
#!pip install livelossplot
!pip install datasets
#!pip install -q transformers[torch]
#!pip install accelerate -U
!pip install peft

# Python utils
#!pip install -q ipdb      # debugging.
#!pip install -q colorama  # print colors :).

import os
import math
import urllib.request

import chex
import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
import optax

import transformers
from transformers import pipeline, AutoTokenizer, AutoModel
import datasets
import peft

from PIL import Image
from livelossplot import PlotLosses
import evaluate

# Utils.
import colorama

import torch
#import torchvision

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import itertools
import random

import copy

import gensim
from nltk.data import find
import nltk


from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup,get_cosine_with_hard_restarts_schedule_with_warmup,get_constant_schedule
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import re
import string
import unicodedata
import re

import huggingface_hub
import ipywidgets as widgets
from IPython.display import display

ERROR: unknown command "accelerate"


[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


# Function definitions

#### Function to load the BLOOM pretrained model from Hugging.

In [2]:
def load_pretrained_model(device: str):
    model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b",
                                             torch_dtype = torch.float32,
                                             device_map = device)
    tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b", padding_side = "left")
    tokenizer.pad_token = tokenizer.eos_token
    # Return the pretrained model and tokenizer for the model.
    return model, tokenizer

#### This utility function prints all trainable parameters of a given model.

In [3]:
def print_trainable_parameters(model):
  trainable_params = 0
  all_params = 0
  for _, param in model.named_parameters():
    all_params += param.numel()
    if(param.requires_grad):
      trainable_params += param.numel()
  print(f"Trainable params: {trainable_params} || All params: {all_params} || Trainable %: {100 * trainable_params / all_params}")

Here we define functions to create a prompt string, and another function to map the input and targets to a prompt string. We want to format our prompts in the form ###INPUT: {input} TARGET {target} for the model, to make it easy for the model to learn where the input is, and where the target is.

In [4]:
def create_prompt(inputs, targets):
    prompt_template = f"""###INPUT: {inputs}
###TARGET: {targets}</s>"""

    return prompt_template

def map_dataset_to_prompt(dataset, max_length):
    ds = dataset.map(lambda example: {'inputs': example['inputs'], 'targets': example['targets']}, remove_columns=list(dataset.features.keys()))
    mapped_dataset = ds.map(lambda samples: tokenizer(create_prompt(samples["inputs"], samples["targets"]), max_length = max_length, truncation = True))
    return mapped_dataset

Here, we define the hyperparameters for training.

In [5]:
batch_size = 1
gradient_accumulation_steps = 4
warmup_steps = 50
max_steps = 50
LR = 1e-3
logging_steps = 1
output_dir = "output"

In [6]:
def train_model(tokenized_dataset):
    trainer = transformers.Trainer(model = model,
                                    train_dataset = tokenized_dataset,
                                    args = transformers.TrainingArguments(per_device_train_batch_size = batch_size,
                                                                          gradient_accumulation_steps = gradient_accumulation_steps,
                                                                          warmup_steps = warmup_steps,
                                                                          max_steps = max_steps,
                                                                          learning_rate = LR,
                                                                          fp16 = False,
                                                                          logging_steps = logging_steps,
                                                                          num_train_epochs = 10,
                                                                          output_dir = output_dir),
                                    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)
                                    )
    model.config.use_cache = False #Silence warnings
    trainer.train()
    return model

Function to decode generated text from the model.

In [51]:
def make_inference(inputs):
    template = f"""###INPUT: {inputs}
###TARGET: """
    batch = tokenizer(template, return_tensors = "pt", return_token_type_ids = False, padding=True).to("cuda")#, return_token_type_ids = False, padding = True).to("cuda")
    with torch.amp.autocast('cuda'):
        output_tokens = model.generate(**batch, max_new_tokens = 128, do_sample = True, top_k = 512, temperature = 0.9)#top_p = 0.1)
    prediction = tokenizer.decode(output_tokens[0], skip_special_tokens = True)
    return prediction

Compute the chrF++ score.

In [8]:
def compute_chrF_plus_plus(predictions, reference):
    chrf = evaluate.load("chrf")
    results = chrf.compute(predictions=predictions, references = reference)
    return results

def extract_prediction(model_output):
    prediction = ""
    segments = model_output.split("###TARGET: ")
    if len(segments) > 1:
        prediction = segments[1].strip()
    return prediction

In [9]:
def Bertscore(output,reference,tokenizer, model):

    inputs1 = tokenizer(output, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(reference, return_tensors="pt", padding=True, truncation=True)

    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

    embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

    similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

    #print("Similarity between the texts: {:.4f}".format(similarity[0][0]))

    return similarity[0][0]

In [10]:
def generate(eval_dataset):
    i = 0
    chrF_score = []
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    bert_model = BertModel.from_pretrained("bert-base-uncased")
    bert_scores = []

    for zd in eval_dataset:
        prediction_list = []
        reference_list = []
        target = zd['inputs']
        prediction = make_inference(target)
        bert_scores.append(Bertscore(prediction, target, bert_tokenizer, bert_model))
        #print(Bertscore_v2(prediction,target))
        prediction_list.append(extract_prediction(prediction))
        temp = []
        temp.append(target)
        reference_list.append(temp)
        chrF_score.append(compute_chrF_plus_plus(prediction_list, reference_list).get("score"))
        if i < 8:
            prediction = prediction.replace("###INPUT:", "### INPUT:\n").replace("###TARGET:", "### RESPONSE:\n")
            display(Markdown(prediction))
        else:
            break
        i += 1
        # if i == 8:
        #    break
    return sum(chrF_score) / len(chrF_score), sum(bert_scores) / len(bert_scores)

Load our pretrained model and tokenizer.

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
model, tokenizer = load_pretrained_model(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
print_trainable_parameters(model)

Trainable params: 3002557440 || All params: 3002557440 || Trainable %: 100.0


# CHRF++ score

In [14]:
def chrfpp_score(output,reference):
  output=[output]
  reference=[[reference]]
  chrf = evaluate.load("evaluate-metric/chrf")
  result = chrf.compute(predictions=output,
                         references=reference,
                         word_order=2,
                         lowercase=True)
  return result


# LoRA

## Load our datasets.

In [15]:
from datasets import concatenate_datasets
# Load the annotations dataset

aya_dataset = load_dataset("CohereForAI/aya_dataset", split='train')
def filter_by_language(example):
    return example['language'] == "English"

# Filter rows based on language
train_dataset = aya_dataset.filter(filter_by_language)
train_dataset = map_dataset_to_prompt(train_dataset, 256)
#print(tokenizer.decode(train_dataset[0]['input_ids']))
train_dataset = train_dataset.remove_columns(list(['inputs', 'targets']))

In [16]:
def filter_by_language2(example):
    return example['language'] == "Zulu"

zul_dataset = aya_dataset.filter(filter_by_language2)
zul_dataset = map_dataset_to_prompt(zul_dataset, 1024)
zul_dataset = zul_dataset.remove_columns(list(['inputs', 'targets']))
final_dataset = concatenate_datasets([train_dataset, zul_dataset])

print(zul_dataset)
print(train_dataset)
print(final_dataset)

Map:   0%|          | 0/1833 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1833
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3944
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5777
})


We print one of the training examples in the Zulu language to have an idea about the length of strings we are dealing with. As you can see from the output, the input and target strings are very long, hence we chose the max_length argument to the tokenizer to be 1024.

In [17]:
print(tokenizer.decode(train_dataset[20]['input_ids']))

###INPUT: What is light reflection?
###TARGET: Light reflection is the process by which light waves bounce off a surface. When light encounters a reflective surface, such as a mirror or smooth water, it changes direction and returns into the medium from which it originated, instead of being absorbed by the surface. This phenomenon follows the law of reflection, which states that the angle of incidence (the angle at which the light hits the surface) is equal to the angle of reflection (the angle at which the light is reflected). Reflection is responsible for many everyday visual experiences, including seeing objects in a mirror or the shimmering of light on water.</s>


## Prompt the pretrained model.

Here we try to prompt the pretrained model to observe the output before fine tuning. We expect to improve the output quality after fine tuning.

In [ ]:
print("What is matter?")
batch = tokenizer("What is matter?", return_tensors = "pt", return_token_type_ids = False).to("cuda")#, padding = True).to("cuda")
with torch.amp.autocast('cuda'):
    output_tokens = model.generate(**batch, max_new_tokens = 256, num_beams = 1, do_sample = True, temperature = 2.0)#, top_k = 2048, top_p = 0.1)
prediction = tokenizer.decode(output_tokens[0], skip_special_tokens = True)
print(f"\n{prediction}")

What is matter?

What is matter? One important type that matter refers to something that happens at all – but which only involves us is thought about as reality as only there are concrete realities. By talking about things you, as individuals have, or thoughtful reflection on some point you, for example, in fact. The latter requires that all beings were a thing so for me being like myself does give out the thought as being like I (meaning we cannot and therefore never feel). From my stand as man to be an earthling I would never have been like as it was (although what was like and for me might not be different). A man with eyes from a distance will always have some similar point in him which you do think might be interesting for you as well to know than in order to be you are someone completely separate who is, also, as everyone you see to have things in common without feeling a bit ony why or ony with this in mind being a real Earthbound.
Even your actions as a member of that I are or 

In [18]:
print(model)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
 

## Configuration of LoRA and training.

In [53]:
config = LoraConfig(r = 128,
                    lora_alpha = 16,
                    target_modules = ['query_key_value'],
                    lora_dropout = 0.1,
                    bias = "lora_only",
                    task_type = "CAUSAL_LM")
model = get_peft_model(model, config)
print_trainable_parameters(model)
model = train_model(train_dataset)

Trainable params: 39552000 || All params: 3041879040 || Trainable %: 1.3002489408651832


Step,Training Loss
1,10.210900
2,11.617200
3,13.785900
4,12.114500
5,10.140700
6,7.493000
7,15.758000
8,7.633800
9,10.525900
10,9.572600


In [20]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 2560)
        (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-29): 30 x BloomBlock(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=7680, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=7680, bias=False)
                )
                (lora_

In [21]:
print_trainable_parameters(model)

Trainable params: 19891200 || All params: 3022218240 || Trainable %: 0.6581655731122845


## Save our model locally in Colab.

In [22]:
model_name = "bloom_for_zul"

In [23]:
model.save_pretrained(model_name, safe_serialization = False)

In [24]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3944
})

## Load our saved model

In [ ]:
pre_model, tokenizer = load_pretrained_model(device)
model = PeftModel.from_pretrained(pre_model, model_name)
model = model.to(device)

## Load evaluation dataset

In [25]:
def filter_by_language_code(example):
    return example['language'] == "eng"
# Filter rows based on language
eval_dataset = load_dataset("CohereForAI/aya_evaluation_suite", "dolly_machine_translated", split = "test")
eval_dataset = eval_dataset.filter(filter_by_language_code)

In [26]:
print(eval_dataset)

Dataset({
    features: ['id', 'inputs', 'targets', 'language', 'script', 'source_id'],
    num_rows: 200
})


## Finally Generate.
Here, we call generate() to generate text for sample inputs. The function also computes the average chrF++ score for all 200 test cases.

In [54]:
generate(eval_dataset)

### INPUT:
 What are 5 ways to eat apples?
### RESPONSE:
  Prick fruit regularly. Cut fruit into halves. Poke a fork or a knife thru the flesh. Bend. Roll the fruit in homemade jelly. Fill the fruit with fruit compote or nuts. Eat directly from the tree, pruning the tree monthly.

Fruit - Prick. Fill with jelly and nuts
Bend. Roll in jelly
Prune. Fill fruit
Eat from tree

Fig: Prick. Fill with fruit compote
Prick. Fill with fruit
Prick. Fill fruit
Eat from tree.

Question:
How is it possible for a person to eat some fruits

### INPUT:
 How to escape from a helicopter trapped in water ?
### RESPONSE:
  Surprise the helicopter in the water and release it from the tail. Follow the trajectory of water to avoid obstructions, and be careful of firewire. Easy!
[Thanks]: Thanks a lot, but this isn't really a solution! Here's what I think needs to be done instead.
1.Surprise vehicle
2.Control the vehicle
3.Surprise helicopter
4.Release vehicle from the tail
5.Follow trajectory of water
6.Be careful of firewire

NOTE: I want to avoid the danger of helicopter catching firewire, but unfortunately, this solution can't be implemented. What are the other possible solutions?
*

### INPUT:
 Provide a list of ways musicians can earn income from their music craft.
### RESPONSE:
  Some musicians earn from selling music in physical form, such as CDs or LPs:  Independent musicians are those who do not have a long-established retail presence. Typically they can't afford to hire professional engineers or musicians and often have limited equipment and a higher skill level. Their music may not be expected to have a major impact on regular tastes, but their fans are willing to pay for celebrity status. They often make little money from streams; however, they can earn a significant amount through merchandise sales, which may include releasing music on reissue formats.

Augmenting the original work

While streams and downloads enable independent musicians

### INPUT:
 How can you open a can without using your hands?
### RESPONSE:
  Use your fingers or a sharp tool to prick the can open with a double-prick. Place the can safely on a flat surface and use your thumb and index finger to prick the can open. Use your fingers to open with your  2 or 3 fingers and opening guide. When the can has been open, allow the can to sit 1 minute to drain the water. Then close the can again and continue to load as normal. Repeat this process until you've reached the end of the can. Use a sharp tool to prick the can open. Close the can safely and load as normal. Discard the pricked can

### INPUT:
 What are some things that I could gift a curious toddler?
### RESPONSE:
  Movies, puzzles, and toys that make them think creatively and learn what they are looking for. Some things they would love to play with are scissors, gummy bears, magnetic cubes, and CDs. If a toy is a social or social-emotional learning object, make it worth their while and play it with them for a long time.  If it's just a gadget that they want to play with, they can get it.

### INPUT:
 What are some ways I could cheer up my friend that just got dumped?
### RESPONSE:
  Let's start with some positive goodie, right?  When you talk about the good things that a person is doing, it's easier for them to focus on positive qualities.  In our next tip, let's discuss some things that are working for you right now.  You are making a difference in someone else's life, so be positive about that.  Then, when you look at the things that you are doing for your friends that are working, think about how you can do more to make a difference in someone else's life.  And the final note: don't view your relationship with a negative lens.  Positive outlook is easier for you to take on any relationship,

### INPUT:
 Suggest 5 horror films to me
### RESPONSE:
  1. The Blair Witch Project. 2. Don't Look Up. 3. The Ring. 4. The Shining and 5. The Haunting of Hill House. ####
class: FOCUS AND RELATIONSHIP  Question: Task: Verify student's understanding of key concepts covered through the book, in social studies. ####CATEGORY: LESSON OBJECTIVES: Assess student's understanding of key concepts covered through the book, in social studies. #### RESPONSE:
   a. Include goals you set for learning. b. Develop strategies to reach your goals. c. Use strategies effectively. d. Und

### INPUT:
 Can you help me avoid credit card debt?
### RESPONSE:
  Avoid credit card debt by following a responsible financial planning strategy. For example, set up a budget to establish proper financial goals and stick to them. Then set aside funds for emergencies. Keep track of all funds and use them to avoid credit card debt. Money, too, can be a powerful motivator. Be aware of the financial benefits of savings and learn what debt-free living looks like in practice.
 
If you have debt, seek out debt-free living experiences that can show you the potential of debt-free living. If you find yourself too emotionally involved in debt, give extra thought to your decision. Your decision may seem unhealth

(13.73711633169156, 0.7198451558748881)